This code uses positive and negative tweets from the NLTK package and accomadating package functions to tokenize, clean, and build a sentiment analysis model from sample tweets. Afterwards this model will be used to analyze samples of tweets that we pull by key word in real time (hopefully).

In [3]:
import nltk
#nltk.download()
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re, string, random
#from nltk import FreqDist, classify, NaiveBayesClassifier 

In [4]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [5]:
print(positive_tweets[0]) #just to show what is being stored in positive/negative/text

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [6]:
print(text[0])

RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP


In [94]:
#Removes hyperlinks and other special symbols then groups elements of tokenized words together using lemmatization 
def cleanNoise(tokenized, stopWords = ()):
    cleaned = []
    for ele, mark in pos_tag(tokenized):
        ele = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', ele)
        ele = re.sub("(@[A-Za-z0-9_]+)","", ele)
        if mark.startswith("NN"):
            grp = 'n'
        elif mark.startswith('VB'):
            grp = 'v'
        else:
            grp = 'a'
            
         #lemmatizes  
        ele = WordNetLemmatizer().lemmatize(ele,grp)
        
        #Checks to make sure ele is not empty, not a repeat, and not punctuation
        if len(ele)>0:
            if ele not in string.punctuation:
                if ele.lower() not in cleaned:
                    cleaned.append(ele.lower())
    return cleaned


def finalClean(tweets):
    cleanedTweets = []
    for x in tweets: 
        clean = cleanNoise(word_tokenize(x), stopWords)
        cleanedTweets.append(clean)

    stop_words = get_stop_words('en')
    remove = ['rt', 's', '\'s', '\'\'','\'','http', 'of', 'with', 'be', 'much', 'their', 'the', 'boston', 'college', 'bostoncollege', 'bc']
    i = 0
    while i<3:
        for line in cleanedTweets:
            for each in line: 
                if each in stop_words:
                    line.remove(each)
                elif each in remove:
                    line.remove(each)
                elif each.startswith('//'):
                    line.remove(each)
        i= i+1
        
    return cleanedTweets   

In [98]:
stopWords = stopwords.words('english')
test = twitter_samples.tokenized('positive_tweets.json')[0]
print(test)
print(cleanNoise(test,stopWords))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
['#followfriday', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [9]:
def modelIn(cleanInput):
    for x in cleanInput:
        yield dict([y, True] for y in x)

In [10]:
def modelMaker():
    posTweets = twitter_samples.strings('positive_tweets.json')
    negTweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    stopWords = stopwords.words('english')
    
    posTokens = twitter_samples.tokenized('positive_tweets.json')
    negTokens = twitter_samples.tokenized('negative_tweets.json')
    posCleanTokens = []
    negCleanTokens = []
    
    for x in posTokens:
        clean = cleanNoise(x,stopWords)
        posCleanTokens.append(clean)
    for x in negTokens:
        clean = cleanNoise(x,stopWords)
        negCleanTokens.append(x)
        
    #posTrue = {}
    #negTrue = {}
    #for x in posCleanTokens:
     #   posTrue.append([x, True])
    #for x in negCleanTokens:
     #   negTrue.append([x,True])
    posTrue = modelIn(posCleanTokens)
    negTrue = modelIn(negCleanTokens)
    
    posData = []
    negData = []
    
    for x in posTrue:
        posData.append((x, "Positive"))
    for x in negTrue:
        negData.append((x,"Negative"))
    
    posNegData = posData + negData
    
    #print(posNegData[:5])
    random.shuffle(posNegData)

    train_data = posNegData[:7000]
    test_data = posNegData[7000:]
    
    #right now the model uses Naive bayes but if we are going to do collapsed gibbs we will replace 
    #This part and we will also probably have to reformat the input
    classifier = nltk.NaiveBayesClassifier.train(train_data)
    return(classifier)
    

In [11]:
words = ['be','good','saw']
dict([x,True] for x in words)

{'be': True, 'good': True, 'saw': True}

In [12]:
model = modelMaker()

In [13]:
custom_tweet = "I love sunshine and rainbows"
custom_tokens = cleanNoise(word_tokenize(custom_tweet))
print(custom_tweet, model.classify(dict([token, True] for token in custom_tokens)))

I love sunshine and rainbows Positive


In [ ]:
##LDA Twitter Portion

In [38]:
import pandas as pd
import tweepy
from textblob import TextBlob
from gensim import corpora, models
import gensim
from stop_words import get_stop_words

In [39]:
auth = tweepy.OAuthHandler('9dQrv1fgVjfRTw4x1vu9Ld1A9', 'ONppfYqFHhqqNaMMWFploNjWCOdXuiOPVwhaiQlH5a6cx7N8Hs')
auth.set_access_token('1381356481161781251-TuZA3s3iF8qHHYI5ikIMlVmRo4grTm', 'u3QMibznnpzi3ikbR6VDssnqc15TCq5f8gyvofPbtExQ7')
api = tweepy.API(auth)

In [40]:
def get_tweets(hashtags, date, numtweets):

    tagged_tweets = tweepy.Cursor(api.search, q=hashtags, lang="en",since=date, tweet_mode="extended").items(numtweets)

    tweets_df = pd.DataFrame(columns=('Date and Time', 'id', 'source', 'Geo', 'Place', 'Retweets', 'tweet', 'Sentiment Polarity'))
    list_tweets = []
    i = 1
    for tweet in tagged_tweets:

        list_tweets.append(tweet.full_text)
        
        tweets_df.at[i, 'id'] = tweet.id
        tweets_df.at[i, 'source'] = tweet.source
        tweets_df.at[i, 'tweet'] = tweet.full_text
        tweets_df.at[i, 'Geo'] = tweet.geo
        tweets_df.at[i, 'Place'] = tweet.place
        tweets_df.at[i, 'Favorites'] = tweet.favorite_count
        tweets_df.at[i, 'Retweets'] = tweet.retweet_count
        tweets_df.at[i, 'Date and Time'] = tweet.created_at
        analysis = TextBlob(tweet.full_text)
        tweets_df.at[i, 'Sentiment Polarity'] = analysis.sentiment.polarity
        i = i + 1

    tweets_df.to_csv(hashtags+'tweets.csv', encoding='utf-8')
    #print(tweets_df.head())
    #print(list_tweets[0])
    return (list_tweets)



In [42]:
tweets = get_tweets('bostoncollege', '2020-04-01', 200)

In [43]:
pos = 0
neg = 0
for x in tweets:
    custom_tokens = cleanNoise(word_tokenize(x))
    res = model.classify(dict([token, True] for token in custom_tokens))
    if res == 'Positive':
        pos+=1
    else:
        neg +=1
print(pos,neg)

190 10


In [99]:
testTweet = tweets[0]
print(testTweet)
custom_tokens = cleanNoise(word_tokenize(testTweet))
print(custom_tokens, model.classify(dict([token, True] for token in custom_tokens)))

@SJPREP @BostonCollege @SJPrep_College Congratulations! #JesuitEducated
['sjprep', 'bostoncollege', 'sjprep_college', 'congratulations', 'jesuiteducated'] Positive


In [100]:
##Training LDA on Tweets to get Topics

cleanedTweets = finalClean(tweets)
dictionary = corpora.Dictionary(cleanedTweets)
corpus = [dictionary.doc2bow(text) for text in cleanedTweets]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word = dictionary, passes=100)

print(ldamodel.print_topics())

[(0, '0.015*"new" + 0.012*"acc" + 0.012*"season" + 0.012*"series" + 0.012*"secure" + 0.012*"split" + 0.012*"saturday" + 0.012*"doubleheader" + 0.012*"sixth" + 0.012*"win"'), (1, '0.013*"’" + 0.013*"2" + 0.013*"plan" + 0.013*"guest" + 0.013*"gr…" + 0.013*"allow" + 0.013*"invite" + 0.013*"commencement" + 0.013*"call" + 0.013*"remain"'), (2, '0.018*"work" + 0.017*"congratulations" + 0.014*"student" + 0.011*"university" + 0.011*"team" + 0.011*"connell" + 0.011*"unccharlotte" + 0.011*"center" + 0.011*"margot" + 0.010*"recreation"'), (3, '0.023*"acc" + 0.020*"another" + 0.020*"tournament" + 0.020*"pitt_base" + 0.020*"win" + 0.020*"mitch" + 0.020*"myers" + 0.020*"throw" + 0.020*"second-ever" + 0.020*"bid"'), (4, '0.032*"draft" + 0.029*"bulletinbc" + 0.029*"class" + 0.026*"eagles" + 0.026*"see" + 0.026*"huge" + 0.026*"2022" + 0.026*"preview" + 0.026*"multiple" + 0.026*"nfl"'), (5, '0.021*"eagle" + 0.021*"linebacker" + 0.017*"week" + 0.017*"transfer" + 0.017*"portal" + 0.017*"second" + 0.017*"h

In [101]:
##Get new tweets, get topic probability distribution
tweets_new = get_tweets('bostoncollege', '2020-04-01', 200)
cleanedTweets2 = finalClean(tweets_new)
dictionary2 = corpora.Dictionary(cleanedTweets)
corpus2 = [dictionary.doc2bow(text) for text in cleanedTweets2]

newTweet = corpus2[0]
print(cleanedTweets2[0])
vector = ldamodel[newTweet]
print(vector)

['go', 'juco', 'route', 'grab', 'late', 'commitment', 'point', 'guard', 'jaeden', 'zackery', 'chipola', 'jaedenzackery', 'details']
[(0, 0.025011051), (1, 0.025013141), (2, 0.02500295), (3, 0.20823243), (4, 0.34171727), (5, 0.025006436), (6, 0.02500295), (7, 0.27500388), (8, 0.025006892), (9, 0.02500295)]
